### 数据预处理

In [1]:
import pandas as pd
import os
from datetime import datetime

#### 导入数据

In [50]:
df = pd.read_excel('./data/'+os.listdir("./data/")[0]).fillna(value=0)

In [51]:
df.columns

Index(['订单编号', '订单时间', '商品类型', '商品名称', '商品描述', '课程原价(元)', '交易金额(元)', '买家',
       '推广来源', '推广方名称', '平台红包(元)', '用户实付(元)', '支付方式', '第三方支付费用(元)',
       '渠道推广费用(元)', '平台服务费用(元)', '商家实际收入（元）', '交易状态', '交易成功时间', '到账状态',
       '到账时间'],
      dtype='object')

#### 汇总总的收入

In [52]:
df['商家实际收入（元）']=df['平台红包(元)']+df['用户实付(元)']-df['第三方支付费用(元)']-df['渠道推广费用(元)']-df['平台服务费用(元)']

In [53]:
sum_income = df['商家实际收入（元）'].sum()

#### 汇总每天的销售收入

In [54]:
t = df['订单时间'][0]

In [55]:
t

'2018/05/10 20:40'

In [56]:
datetime.strptime(t,"%Y/%m/%d %H:%M")

datetime.datetime(2018, 5, 10, 20, 40)

In [57]:
print(datetime.strptime(t,"%Y/%m/%d %H:%M"))

2018-05-10 20:40:00


In [58]:
df['订单日期'] = df['订单时间'].apply(lambda s:datetime.strptime(s,"%Y/%m/%d %H:%M") .date())

In [59]:
day_income = df.groupby('订单日期')['商家实际收入（元）'].sum()

#### 汇总top10课程收入数据

In [60]:
course_income = df.groupby('商品名称')['商家实际收入（元）'].sum().sort_values().tail(10)
course_income = pd.DataFrame(course_income).reset_index()
course_income

,商品名称,商家实际收入（元）
0,常用试验设计与SPSS数据分析,88.21
1,问卷与量表数据分析,88.21
2,Python基础—数据科学方向,105.21
3,Python可视化分析(Pyecharts),111.45
4,STATA基础教程,128.12
5,SQL数据分析,178.74
6,R语言数据分析,934.67
7,Python数据分析实战,1381.05
8,STATA统计分析,2224.46
9,SPSS与统计分析,2530.67


#### 汇总top10课程销量

In [62]:
course_count = df[df['商品名称'].isin(course_income['商品名称'])].groupby('商品名称')['订单编号'].count()
course_count = pd.DataFrame(course_count).reset_index()
course_count

,商品名称,订单编号
0,Python可视化分析(Pyecharts),2
1,Python基础—数据科学方向,2
2,Python数据分析实战,5
3,R语言数据分析,5
4,SPSS与统计分析,15
5,SQL数据分析,2
6,STATA基础教程,4
7,STATA统计分析,12
8,常用试验设计与SPSS数据分析,1
9,问卷与量表数据分析,1


#### 将销量和销售额合并

In [63]:
course_income_count = pd.merge(course_income,course_count,on = '商品名称',how ='left')
course_income_count.columns = ['课程','销售额','销量']
course_income_count

,课程,销售额,销量
0,常用试验设计与SPSS数据分析,88.21,1
1,问卷与量表数据分析,88.21,1
2,Python基础—数据科学方向,105.21,2
3,Python可视化分析(Pyecharts),111.45,2
4,STATA基础教程,128.12,4
5,SQL数据分析,178.74,2
6,R语言数据分析,934.67,5
7,Python数据分析实战,1381.05,5
8,STATA统计分析,2224.46,12
9,SPSS与统计分析,2530.67,15


In [68]:
x = course_income_count['课程'].tolist()
y = course_income_count['销售额'].tolist()
z = course_income_count['销量'].tolist()

### 可视化展示

#### 总的销售收入展示

In [64]:
from pyecharts import WordCloud

name = ["Total Income: "+str(round(sum_income,0))+'元']

wordcloud = WordCloud("销售收入统计")
value = [1000]
wordcloud.add("", name, value,shape='diamond')
wordcloud.render()
wordcloud

#### 每日课程收入展示

In [65]:
from pyecharts import Line,Bar,Overlap

day = day_income.index.tolist()
income = day_income.values.round().tolist()

line = Line("")
line.add("", day,income, is_smooth=True, line_width=3,mark_line=["max", "average"],label_color=['red'])

bar_income = Bar("每日课程收入")
bar_income.add("",day,income,label_color=['grey'],is_label_show = True)

overlap = Overlap()
overlap.add(bar_income)
overlap.add(line)
overlap

#### 不同课程收入与销量

In [72]:
from pyecharts import Bar,Style

style = Style()

style_bar = style.add(
    legend_top="bottom",        # 图例位置
    yaxis_label_textsize=9,     # y轴标签文字大小
    yaxis_rotate=45,            # y轴标签选择角度
    is_label_show = True,       # 展示柱形图上面的数值标签
    label_pos="right",          # 柱形图上面的数值标签显示位置
    label_text_size=12          # 数值标签文本大小
)

course_income = df.groupby('商品名称')['商家实际收入（元）'].sum().sort_values().tail(10)
course_income = pd.DataFrame(course_income).reset_index()

course_count = df[df['商品名称'].isin(course_income['商品名称'])].groupby('商品名称')['订单编号'].count()
course_count = pd.DataFrame(course_count).reset_index()

course_income_count = pd.merge(course_income,course_count,on = '商品名称',how ='left')
course_income_count.columns = ['课程','销售额','销量']

x = course_income_count['课程'].tolist()
y = course_income_count['销售额'].round().tolist()
z = course_income_count['销量'].tolist()

bar = Bar("单课销售量与销售额统计","",height=400)
bar.add("销售额",x,y,is_convert=True,yaxis_index=0,**style_bar)
bar.add("销售量",x,z,is_convert=True,yaxis_index=1,**style_bar)
bar.render()
bar

#### 将上面的图形展示在一起

In [51]:
from pyecharts import Page

page = Page()
page.add(wordcloud)
page.add(overlap)
page.add(bar)
page.render('./test01.html')

### 将代码整合到一起

#### 整理代码

将重复的部分去除，整理成一个完整的代码块

In [73]:
import pandas as pd
import os
from datetime import datetime
from pyecharts import Line,Page,WordCloud,Bar,Style,Overlap

# 数据预处理
df = pd.read_excel('./data/'+os.listdir("./data/")[0]).fillna(value=0)
df['商家实际收入（元）'] = df['平台红包(元)'] + df['用户实付(元)'] - df['第三方支付费用(元)'] - df['渠道推广费用(元)'] - df['平台服务费用(元)']

# 计算销售总收入
sum_income = df['商家实际收入（元）'].sum()

# 汇总每日销售收入
df['订单日期'] = df['订单时间'].apply(lambda s:datetime.strptime(s,"%Y/%m/%d %H:%M") .date())
day_income = df.groupby('订单日期')['商家实际收入（元）'].sum()

# 汇总top10课程收入
course_income = df.groupby('商品名称')['商家实际收入（元）'].sum().sort_values().tail(10)
course_income = pd.DataFrame(course_income).reset_index()

# 收入top10课程销量
course_count = df[df['商品名称'].isin(course_income['商品名称'])].groupby('商品名称')['订单编号'].count()
course_count = pd.DataFrame(course_count).reset_index()

# 收入和销量数据合并
course_income_count = pd.merge(course_income,course_count,on = '商品名称',how ='left')
course_income_count.columns = ['课程','销售额','销量']

x = course_income_count['课程'].tolist()
y = course_income_count['销售额'].round().tolist()
z = course_income_count['销量'].tolist()

# 图表宽度参数
self_width = 900

# 总收入
name = ["Total Income: "+str(round(sum_income,0))+'元']
value = [1000]
wordcloud = WordCloud("总的销售收入统计",width = self_width,height=150)
wordcloud.add("", name, value,shape='diamond')


# 每日收入
day = day_income.index.tolist()
income = day_income.values.round().tolist()
# 折线图
line = Line("每日课程收入")
line.add("", day,income, is_smooth=True, line_width=3,mark_line=["max", "average"],label_color=['red'])
# 柱形图
bar_income = Bar("每日课程收入")
bar_income.add("",day,income,label_color=['grey'],is_label_show = True)

# 整合每日收入折线图与柱形图
overlap = Overlap(width = self_width,height=350)
overlap.add(bar_income)
overlap.add(line)


# 声明style
style = Style()

# 柱形图外观设置
style_bar = style.add(
    legend_top="bottom",        # 图例位置
    yaxis_label_textsize=9,     # y轴标签文字大小
    yaxis_rotate=45,            # y轴标签选择角度
    is_label_show = True,       # 展示柱形图上面的数值标签
    label_pos="right",          # 柱形图上面的数值标签显示位置
    label_text_size=12          # 数值标签文本大小
)

bar = Bar("单课销售量与销售额统计",width = self_width,height=400)
bar.add("销售额",x,y)
bar.add("销售量",x,z,is_convert=True,**style_bar)


# 组合输出
page = Page()
page.add(wordcloud)
page.add(overlap)
page.add(bar)
page.render('./test02.html')

#### 将代码应用其它时期的销售订单数据

将data目录下的路径替换成其它数据集，运行代码即可。